In [1]:
pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_sm-0.5.4.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 4.1 MB/s eta 0:00:0000:0100:01m
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for spacy<3.8.0,>=3.7.4 from https://files.pythonhosted.org/packages/72/49/bd65abe76607c86dc1f104ad545eeb3e771f474b7e259e64e5a16614615b/spacy-3.7.5-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for spacy-legacy<3.1.0,>=3.0.11 from https://files.pythonhosted.org/packages/c3/55/12e842c70ff8828e34e543a2c7176dac4da006ca6901c9e8b43efab8bc6b/spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata
  Obtaining dependency information for spacy-loggers<2.0.0,>=1.0.0 from https://files.pythonhosted.org/packages/33/78/d1a1a026ef3af911159398c939b1509d5c36fe524c7b644f34a5146c4e16/spacy_loggers-1.0.5-py3-none-any.whl.metadata
  Obtaining dependency information for murmurhash<1.1.0,>=0.28.0 from https://files.pythonhosted.org/packages/2f/a4/a387486e79bcc04f3d3b123195fd4cca74a7ba439d6c45b35c5366c66586/murmurhash-

In [3]:
pip install datasets scispacy

Note: you may need to restart the kernel to use updated packages.


Login into the hg account because otherwise you wont get access to the datasets

In [6]:
from huggingface_hub import login
login("eheh")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.


ValueError: Invalid token passed!

Get all the imports

In [7]:
from datasets import load_dataset
import os
import json
import pickle
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from datasets import load_dataset
import spacy
import cupy
import scispacy
from scispacy.linking import EntityLinker

ModuleNotFoundError: No module named 'cupy'

# 1) GET THE DATASETS
Define all the datasets names and other global variables if needed

In [ ]:
# Datasets' names
TESTDATASET = "openlifescienceai/medmcqa"
REPO_ID = "MGFiD-Group/MedMCQA_utils"
METADATANAME = "database_w_metadata"
RETRIEVALINFONAME = "retrieval_infos"

TOPK_DOCUMENTS = 5
#MAX_MD_NODE_PER_QUESTION = 1000
#MAX_QAABST_NODE_PER_QUESTION = 1000
FNAMES = ["train"]
FNAME = 'train'

repo_root = '..'
medqa_root = f'{repo_root}/data/{TESTDATASET}'
# os.system(f'mkdir -p {medqa_root}/statement')
# os.system(f'mkdir -p {repo_root}/data/{TESTDATASET}/metadata')
# os.system(f'mkdir -p {repo_root}/data/{TESTDATASET}/embeddings')

Load the 3 datasets

In [8]:
'''
The medqa dataset
'''
qa_ds = load_dataset(TESTDATASET)

'''
The pubmed dataset
'''
db_w_metadata = load_dataset(REPO_ID, METADATANAME)#, use_auth_token=True)

'''
Custom dataset that connects the previous two.
- ID
  The enum index of each entry of the medqa dataset
- retrieval_relative_indices
  A list of PMIDs which are relevant to the question
'''
retr_info_ds = load_dataset(REPO_ID, RETRIEVALINFONAME)#, use_auth_token=True)

NameError: name 'TESTDATASET' is not defined

In [9]:
display(qa_ds)
print("\n\n")
display(db_w_metadata)
print("\n\n")
display(retr_info_ds)

NameError: name 'qa_ds' is not defined

In [11]:
# prompt: Create a datasetdict called qa_ds_reduced which will have 3 splits: train, test and validation train will have the first 500 entries of qa_ds['train'], test will have the first 100 entrie of qa_ds['test'] and validation the first 70 entries of qa_ds['validation']

qa_ds_reduced = {
    'train': qa_ds['train'].select(range(500)),
    'test': qa_ds['test'].select(range(100)),
    'validation': qa_ds['validation'].select(range(70))
}


In [13]:
# prompt: Create a DatasetDict() called qa_ds_reduced which will have 3 splits: train, test and validation train will have the first 500 entries of qa_ds['train'], test will have the first 100 entrie of qa_ds['test'] and validation the first 70 entries of qa_ds['validation']

from datasets import DatasetDict

qa_ds_reduced = DatasetDict({
    'train': qa_ds['train'].select(range(500)),
    'test': qa_ds['test'].select(range(100)),
    'validation': qa_ds['validation'].select(range(70))
})


In [14]:
display(qa_ds_reduced)

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 100
    })
    validation: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 70
    })
})

In [15]:
# prompt: Now i want to save the dataset qa_ds_reduced as json that i can later re-load
qa_ds_reduced.save_to_disk("qa_ds_reduced.hf")



Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/70 [00:00<?, ? examples/s]

In [16]:
!zip -r /content/qa_ds_reduced.zip /content/qa_ds_reduced.hf

  adding: content/qa_ds_reduced.hf/ (stored 0%)
  adding: content/qa_ds_reduced.hf/validation/ (stored 0%)
  adding: content/qa_ds_reduced.hf/validation/data-00000-of-00001.arrow (deflated 56%)
  adding: content/qa_ds_reduced.hf/validation/state.json (deflated 38%)
  adding: content/qa_ds_reduced.hf/validation/dataset_info.json (deflated 75%)
  adding: content/qa_ds_reduced.hf/test/ (stored 0%)
  adding: content/qa_ds_reduced.hf/test/data-00000-of-00001.arrow (deflated 53%)
  adding: content/qa_ds_reduced.hf/test/state.json (deflated 37%)
  adding: content/qa_ds_reduced.hf/test/dataset_info.json (deflated 75%)
  adding: content/qa_ds_reduced.hf/train/ (stored 0%)
  adding: content/qa_ds_reduced.hf/train/data-00000-of-00001.arrow (deflated 58%)
  adding: content/qa_ds_reduced.hf/train/state.json (deflated 38%)
  adding: content/qa_ds_reduced.hf/train/dataset_info.json (deflated 75%)
  adding: content/qa_ds_reduced.hf/dataset_dict.json (deflated 2%)


# 2) PARSE THE MEDQA DATASET & PREPARE THE PUBMED ARTICLE

First, reformat the dataset to mainly get the options bettwe deligneated and to get the statements.
Statements are just the question followed by each of the possible answers (thus 4 entries)

In [ ]:
options_map = {'A' : 'opa', 'B' : 'opb', 'C' : 'opc', 'D' : 'opd'}

for fname in ['train', 'test', 'validation']:
    #with open(f"{medqa_root}/raw/questions/US/4_options/phrases_no_exclude_{fname}.jsonl") as f:
        #lines = f.readlines()
    examples = []
    for i in tqdm(range(len(qa_ds[fname]))):
        line = qa_ds[fname][i]
        _id  = f"{fname}-{i:05d}"
        answerKey = line["cop"]
        stem      = line["question"]
        #explanation = line["exp"]
        #choice_type = line["choice_type"]
        #subject_name = line["subject_name"]
        #topic_name = line["topic_name"]
        choices   = [{"label": choice, "text": line[options_map[choice]]} for choice in options_map]
        stmts     = [{"statement": stem +" "+ c["text"]} for c in choices]
        ex_obj    = {"id": _id,
                     "question": {"stem": stem, "choices": choices},
                     "answerKey": answerKey,
                     "statements": stmts,
                     #"explanation": explanation,
                     #"choice_type": choice_type,
                     #"subject_name": subject_name,
                     #"topic_name": topic_name,
                    }
        examples.append(ex_obj)
    with open(f"{medqa_root}/statement/{fname}.statement.jsonl", 'w') as fout:
        for dic in examples:
            print (json.dumps(dic), file=fout)

Now, with the below methods, it wil create two long ass files:
- `"{medqa_root}/metadata/{fname}_ptrs.txt"`

  it contains all the (unique wrt retr_info_ds) PMIDs

- `"{medqa_root}/metadata/{fname}_vocabs.txt"`

  it contains all the (unique wrt retr_info_ds) abstracts

These two file are thus linked
id-abstract

In [ ]:
'''
Turn a dict of lists into a list of dicts
e.g.:
dict_of_lists = {
    "name": ["Alice", "Bob", "Charlie"],
    "age": [30, 25, 35],
    "city": ["New York", "Los Angeles", "Chicago"]
}
to
[{'name': 'Alice', 'age': 30, 'city': 'New York'},
 {'name': 'Bob', 'age': 25, 'city': 'Los Angeles'},
 {'name': 'Charlie', 'age': 35, 'city': 'Chicago'}]

This is to handle what comes out from treat_topk_articles_as_unique_articles
which has this format:
{'PMID': [32665527, 8599280, 12822472, 30647141],
 'Year': ['2021', '1996', '2003', '2019'],
'''
def dict_of_lists_to_list_of_dicts(dict_of_lists):
    # Get all keys and their corresponding lists
    keys = list(dict_of_lists.keys())
    lists = list(dict_of_lists.values())
    # Check if all lists are of the same length
    length = len(lists[0])
    if not all(len(lst) == length for lst in lists):
        raise ValueError("All lists must be of the same length")

    # Use zip to group elements and create the list of dictionaries
    return [dict(zip(keys, values)) for values in zip(*lists)]


'''
Transforms a list of lists into as single chonky set, thus obtain a single set
without duplicates.
e.g.:
[[1,3],[4,3],[44,1]]
old elements: 6, new elements: 4
{1, 3, 4, 44}
'''
def turn_a_list_of_lists_into_a_set(lst):
    # Flatten the list of lists into a single list
    new_lst = [x for xs in lst for x in xs]
    # Convert the list to a set to remove duplicates
    unique_lst = set(new_lst)
    print(f"old elements: {len(new_lst)}, new elements: {len(unique_lst)}")
    return unique_lst


'''
From the retr_info_ds dataset, get the top-K document PMIDs for a given question
enum ID (not the uuid of the question)
'''
# Get the top-K document IDs for a given question ID
def get_topk_ids_per_question_id(qa_id):
    return retr_info_ds[FNAME]['retrieval_relative_indices'][qa_id][:TOPK_DOCUMENTS]


'''
From the db_w_metadata dataset, obtain all the pubmed articles (unique, filtered
out) that are connected to the questions found in retr_info_ds
'''
# Function to treat top K articles as unique articles
def treat_topk_articles_as_unique_articles(fname):
    # Get the list of article indexes for each question
    articles_indexes = retr_info_ds[fname]['retrieval_relative_indices']
    # Remove duplicate articles
    unique_articles_indexes = turn_a_list_of_lists_into_a_set(articles_indexes)
    # Filter out invalid article indexes (-1)
    unique_articles_indexes = list(filter(lambda x: x != -1, unique_articles_indexes))
    # Return the unique articles from the metadata dataset
    return db_w_metadata[fname][unique_articles_indexes]

In [ ]:
#create unique document nodes in txts, so we also have unique index.
for fname in FNAMES:
    articles = treat_topk_articles_as_unique_articles(fname)
    #articles = dict_of_lists_to_list_of_dicts(articles)

    with open(f"{repo_root}/data/{TESTDATASET}/metadata/{fname}_ptrs.txt", "w") as fout:
        for doc in tqdm(articles['PMID']):
            print (doc, file=fout)

    with open(f"{repo_root}/data/{TESTDATASET}/metadata/{fname}_vocabs.txt", "w") as fout:
        for doc in tqdm(articles['Abstract']):
            print (doc, file=fout)

# 3) THE NER AND THE LINKER
Create the ner model (nlp) and the entity linker (linker)

In [ ]:
def load_entity_linker(threshold=0.90):
    """
    Load the SciSpacy entity linker with a specified threshold.

    Args:
        threshold (float): The threshold for entity linking. Default is 0.90.

    Returns:
        nlp (spacy.Language): The loaded SpaCy model with the entity linker.
        linker (scispacy.linking.EntityLinker): The entity linker component.
    """
    # Load the SciSpacy model
    nlp = spacy.load("en_core_sci_sm")

    # Add the entity linker to the pipeline
    nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls", "threshold": threshold})

    # Return the loaded model and the entity linker
    return nlp, nlp.get_pipe("scispacy_linker")

# Load the entity linker with the default threshold
nlp, linker = load_entity_linker()

/usr/local/lib/python3.11/dist-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linkers/2023-04-23/umls/tfidf_vectors_sparse.npz not found in cache, downloading to /tmp/tmpmczuxjh1


100%|██████████| 492M/492M [00:11<00:00, 45.6MiB/s]


Finished download, copying /tmp/tmpmczuxjh1 to cache at /root/.scispacy/datasets/2b79923846fb52e62d686f2db846392575c8eb5b732d9d26cd3ca9378c622d40.87bd52d0f0ee055c1e455ef54ba45149d188552f07991b765da256a1b512ca0b.tfidf_vectors_sparse.npz
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linkers/2023-04-23/umls/nmslib_index.bin not found in cache, downloading to /tmp/tmpny1etge1


100%|██████████| 724M/724M [00:17<00:00, 42.7MiB/s]


...

Finished download, copying /tmp/tmprfooudft to cache at /root/.scispacy/datasets/6238f505f56aca33290aab44097f67dd1b88880e3be6d6dcce65e56e9255b7d4.d7f77b1629001b40f1b1bc951f3a890ff2d516fb8fbae3111b236b31b33d6dcf.concept_aliases.json
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/kbs/2023-04-23/umls_2022_ab_cat0129.jsonl not found in cache, downloading to /tmp/tmpqidn2vba


100%|██████████| 628M/628M [00:16<00:00, 39.4MiB/s]


Finished download, copying /tmp/tmpqidn2vba to cache at /root/.scispacy/datasets/d5e593bc2d8adeee7754be423cd64f5d331ebf26272074a2575616be55697632.0660f30a60ad00fffd8bbf084a18eb3f462fd192ac5563bf50940fc32a850a3c.umls_2022_ab_cat0129.jsonl
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/umls_semantic_type_tree.tsv not found in cache, downloading to /tmp/tmp2wca8eny


100%|██████████| 4.26k/4.26k [00:00<00:00, 5.89MiB/s]

Finished download, copying /tmp/tmp2wca8eny to cache at /root/.scispacy/datasets/21a1012c532c3a431d60895c509f5b4d45b0f8966c4178b892190a302b21836f.330707f4efe774134872b9f77f0e3208c1d30f50800b3b39a6b8ec21d9adf1b7.umls_semantic_type_tree.tsv


  # 4) START TO TRASNFORM THE ENTRIES TO THE KG FORMAT
  
  Now with the below methods, re-opens the `"{medqa_root}/metadata/{fname}_vocabs.txt"` with the abstracts and, for each one of them, it performs the entity linking process, saving the results in `"{medqa_root}/metadata/{fname}_vocabs_ulms_entities.txt"`

  For each abstract it will pass through the NER model `nlp` and for each found entry it will return with the `linker` the linked entities which have a match > .9





In [ ]:
'''
For each entry performs the entity linking process
'''
def process_entities(entities, linker):
    # parse the returned metadata
    all_entities_results = []
    for mm in tqdm(range(len(entities))):
        # get the entity's name (as mentioned in the input text)
        entity_text = entities[mm].text
        entity_start = entities[mm].start
        entity_end = entities[mm].end

        # get the corresponding entity name in the UMLS ontology (for one entity name we may have more than one correspondence in UMLS)
        all_linked_entities = entities[mm]._.kb_ents
        all_entity_results = []
        for ii in range(len(all_linked_entities)):
            curr_concept_id = all_linked_entities[ii][0]
            curr_score = all_linked_entities[ii][1]

            # Get the numerical ID (CUI) of the entity in UMLS
            curr_scispacy_entity = linker.kb.cui_to_entity[all_linked_entities[ii][0]]
            curr_canonical_name = curr_scispacy_entity.canonical_name
            curr_TUIs = curr_scispacy_entity.types
            # store the resulting metadata
            curr_entity_result = {"Canonical Name": curr_canonical_name, "Concept ID": curr_concept_id,
                                  "TUIs": curr_TUIs, "Score": curr_score}
            all_entity_results.append(curr_entity_result)
        curr_entities_result = {"text": entity_text, "start": entity_start, "end": entity_end,
                                "start_char": entities[mm].start_char, "end_char": entities[mm].end_char,
                                "linking_results": all_entity_results}
        all_entities_results.append(curr_entities_result)
    return all_entities_results


'''
Performs process_entities for each abstract
'''
def entity_linking_to_umls_from_a_batch(sentences, nlp, linker):
    doc = nlp.pipe(sentences)
    return [process_entities(element.ents, linker) for element in doc]

In [ ]:
for fname in FNAMES:
    abstracts_texts = []
    abstracts_texts_ners = []
    with open(f"{repo_root}/data/{TESTDATASET}/metadata/{fname}_vocabs.txt") as fin:
        for line in tqdm(fin):
            abstracts_texts.append(line)

    with open(f"{repo_root}/data/{TESTDATASET}/metadata/{fname}_vocabs_ulms_entities.txt", "w") as fout:
        abstracts_texts_ners = entity_linking_to_umls_from_a_batch(abstracts_texts, nlp, linker)
        json.dump(abstracts_texts_ners, fout, ensure_ascii=False)


Now it performs the same process but for each one of the questions and their possible answers

The result will be saved in `"{medqa_root}/statement/{fname}.statement.umls_linked.jsonl"`

In [ ]:
# Run entity linking to UMLS for all questions
def process_question_answer_entities(input):
    nlp, linker = load_entity_linker()
    stmts = input
    for stmt in tqdm(stmts):
        stem = stmt['question']['stem']
        stem = stem[:3500]
        stmt['question']['stem_ents'] = entity_linking_to_umls(stem, nlp, linker)
        for ii, choice in enumerate(stmt['question']['choices']):
            text = stmt['question']['choices'][ii]['text']
            stmt['question']['choices'][ii]['text_ents'] = entity_linking_to_umls(text, nlp, linker)
    return stmts

In [ ]:
for fname in ["validation", "test", "train"]:
    with open(f"{medqa_root}/statement/{fname}.statement.jsonl") as fin:
        stmts = [json.loads(line) for line in fin]
        res = process_question_answer_entities(stmts)
    with open(f"{medqa_root}/statement/{fname}.statement.umls_linked.jsonl", 'w') as fout:
        for dic in res:
            print (json.dumps(dic), file=fout)

Now, it will obtain:

- `md_concept2id`

  A long ass dict with all the PMIDs that were present in `"{medqa_root}/metadata/{fname}_ptrs.txt"`

  More specifically it will have entries in the format "PMID": enum_index

  e.g.: 12345: 2 will mean that the PMID=12345 will be the second entry

- `md_vocab_entities`

  A long ass list with all the entries being all the entity linked abstract structured saved in `"{medqa_root}/metadata/{fname}_vocabs_ulms_entities.txt"`

So entry 2 of `md_concept2id` is linked to `md_vocab_entities` and so on for each and every one.



In [ ]:
def remove_newlines_from_list(input_list):
    return [s.strip() for s in input_list]


def get_article_umls_entities(fname):
    md_vocab_entities = []
    with open(f"{repo_root}/data/{TESTDATASET}/metadata/{fname}_vocabs_ulms_entities.txt") as fin:
        for val in json.load(fin):
            md_vocab_entities.append(val)
    return md_vocab_entities


def get_article_pointers(fname):
    md_id2concept = []
    with open(f"{repo_root}/data/{TESTDATASET}/metadata/{fname}_ptrs.txt") as fin:
        lines = fin.readlines()
        md_id2concept = [l for l in remove_newlines_from_list(lines)]
    return md_id2concept

In [ ]:
md_id2concept = get_article_pointers(FNAME)
md_concept2id = {int(w):i for i, w in enumerate(md_id2concept)}

md_vocab_entities = get_article_umls_entities(FNAME)

Now with the link between PMIDs and their entity linked abstracts it is time to connect them to each question

In the file `"{medqa_root}/statement/{FNAME}.statement.umls_linked_abstract.jsonl"` it will now create a new structure which is just the uninion between the previous one

It will have the same structure as `"{medqa_root}/statement/{FNAME}.statement.umls_linked.jsonl"` but with added at the end all the linked entities of all the abstracts correlated to that qa entry

In [ ]:
# Define the process_stmt function at the top level
def process_stmt(index_stmt_tuple):
    index, stmt = index_stmt_tuple
    pmids = db_w_metadata[FNAME][get_topk_ids_per_question_id(index)]['PMID']
    stmt['abstract_ents'] = list(map(lambda pmid: md_vocab_entities[md_concept2id[pmid]], pmids))
    return stmt

# Example function to process abstract entities in parallel
def process_abstract_entities_parallel(stmts, num_processes=13):
    # Create a Pool of processes
    with Pool(num_processes) as pool:
        # Map the function to statements using the Pool
        processed_stmts = list(tqdm(pool.imap(process_stmt, enumerate(stmts)), total=len(stmts)))

    return processed_stmts

In [ ]:
with open(f"{medqa_root}/statement/{FNAME}.statement.umls_linked.jsonl") as fin:
    stmts = [json.loads(line) for line in fin]
    res = process_abstract_entities_parallel(stmts)
    with open(f"{medqa_root}/statement/{FNAME}.statement.umls_linked_abstract.jsonl", 'w') as fout:
        for dic in res:
            print (json.dumps(dic), file=fout)

# 5) FURTHER LINKING WITH DDB

The idea now is to further restrict the medical ontology using another reference system

Diseases Database + DrugBank (DDB) is a huge pharmaceutical knowledge database. It contains information on drugs, drug targets, and more

For magical reasons we already have a file containing the mapping between UMLS and DDB IDs

First, creates a dictionary which provides a direct lookup from a UMLS CUI to its associated DDB identifiers (ItemPTR)




In [ ]:
umls_to_ddb = {}
with open(f'{repo_root}/data/ddb/ddb_to_umls_cui.txt') as f:
    for line in f.readlines()[1:]:
        elms = line.split("\t")
        umls_to_ddb[elms[2]] = elms[1]

Now for each magical entry in `"{medqa_root}/statement/{fname}.statement.umls_linked_abstract.jsonl"` it will perform the below methods

At the end it will obtain the file `"{medqa_root}/grounded/{fname}.grounded.jsonl"` which should be the "grounded" version of the one above

Grounded in the sense that each entry will have:
- `sent`: The question
- `ans`: The answers
- `qc`: The list of DDB ids concept related to each entity of the question (qc = question concepts(?))
- `qc_names`: The list of names of UMLS relate concepts to each entity of the question
- `ac`: The list of DDB ids concept related to each entity of each answer (qc = answer concepts(?))
- `qac_names`: The list of names of UMLS relate concepts to each entity of each answer
- `abstc`: The list of DDB ids concept related to the each entity of each abstract
- `abstc_names`: The list of names of UMLS relate concepts to each entity of each abstract


In [ ]:
'''
Returns a list res of tuples (ddb ID, name of the UMLS related concept)
'''
def map_to_ddb(ent_obj):
    res = []
    for ent_cand in ent_obj['linking_results']:
        CUI  = ent_cand['Concept ID']
        name = ent_cand['Canonical Name']
        if CUI in umls_to_ddb:
            ddb_cid = umls_to_ddb[CUI]
            res.append((ddb_cid, name))
    return res


'''
For entry it first extracts the question in sent
Then it iterates through each entity linked to that question (under 'stem_ents')
For each entity it will extract:
  - in qc it will append the correlated DDB ID to that specific UMLS entity
  - in qc_names it will append the name of the UMLS concept
it will finally return qc, qc_names, sent
'''
def map_stem_to_ddb(stmt):
    sent = stmt['question']['stem']
    qc = []
    qc_names = []
    for ent_obj in stmt['question']['stem_ents']:
        res = map_to_ddb(ent_obj)
        for elm in res:
            ddb_cid, name = elm
            qc.append(ddb_cid)
            qc_names.append(name)
    return qc, qc_names, sent


'''
For entry it extracts all the possible answers in ans
Then for each answer it will extract all the entities linked to that answer
(under 'text_ents')
For each entity it will extract:
  - in ac it will append the correlated DDB ID to that specific UMLS entity
  - in ac_names it will append the name of the UMLS concept
it will finally return ac, ac_names, ans
'''
def map_answer_to_ddb(stmt):
    acs = []
    acs_names = []
    answers = []
    for cid, choice in enumerate(stmt['question']['choices']):
        ans = choice['text']
        ac = []
        ac_names = []
        for ent_obj in choice['text_ents']:
            res = map_to_ddb(ent_obj)
            for elm in res:
                ddb_cid, name = elm
                ac.append(ddb_cid)
                ac_names.append(name)
        acs.append(ac)
        acs_names.append(ac_names)
        answers.append(ans)
    return acs, acs_names, answers


'''
For each entry it extracts all the connect abstracts
Then for each abstract it will extract all the entities linked to that abstract
(under 'abstract_ents')
For each entity it will extract:
  - in abstc it will append the correlated DDB ID to that specific UMLS entity
  - in abstc_names it will append the name of the UMLS concept
it will finally return abstc, abstc_names
'''
def map_abstracts_to_ddb(stmt):
    abstract_concepts = []
    abstract_concept_names = []
    for abstract in stmt['abstract_ents']:
        abstc = []
        abstc_names = []
        for ent_obj in abstract:
            res = map_to_ddb(ent_obj)
            for elm in res:
                ddb_cid, name = elm
                abstc.append(ddb_cid)
                abstc_names.append(name)
        abstract_concepts.append(abstc)
        abstract_concept_names.append(abstc_names)
    return abstract_concepts, abstract_concept_names




'''
Opne the json with the entity linkings of the question, the possible answers
and all the connected abstracts
Then, for each entry it will perform the above methods creating grounded.jsonl
'''
def process_with_abstracts(fname):
    with open(f"{medqa_root}/statement/{fname}.statement.umls_linked_abstract.jsonl") as fin:
        stmts = [json.loads(line) for line in fin]
    with open(f"{medqa_root}/grounded/{fname}.grounded.jsonl", 'w') as fout:
        for stmt in tqdm(stmts):
            qc, qc_names, sent = map_stem_to_ddb(stmt)
            acs, acs_names, answers = map_answer_to_ddb(stmt)
            abstc, abstc_names = map_abstracts_to_ddb(stmt)
            out = {'sent': sent, 'ans': answers, 'qc': qc, 'qc_names': qc_names, 'ac': acs, 'ac_names': acs_names, 'abstc':abstc, 'abstc_names':abstc_names}
            print (json.dumps(out), file=fout)

In [ ]:
os.system(f'mkdir -p {medqa_root}/grounded')

for fname in FNAMES:
    process_with_abstracts(fname)

# 6) CREATE THE KG OF DDB

Create the kg of the ddb dataset

First extract all the relationship and various names

In [ ]:
'''
Opne the json ddb_names and get all its entries
They have this format
  "Allylbarbital": ["30938", "0"]
- Key => the ddb item name
- val[0] => the ddb ID of the item
- val[1] => if the item key is the preferred name (1) or not (0) for the entry

Open the json ddb_relas and get all its entries
They have this format
  "39": ["9836", "16", "2"]
- Key => general index of each entry
- val[0] => Source DDB ID
- val[1] => Target DDB ID
- val[2] => Relation type

The method returns :
- relas_lst => the list of all the ddb relations
- ddb_ptr_to_name => a dict in the form key:ItemName, value:ItemID(the pointer)
- ddb_name_to_ptr => a dict in the form key:ItemID(the pointer), value:ItemName
- ddb_ptr_to_preferred_name => a dict in the form key:ItemName,
                               value:ItemID(the pointer); this only if the
                               val[1] of the ddb_names is 1
'''
def load_ddb():
    with open(f'{repo_root}/data/ddb/ddb_names.json') as f:
        all_names = json.load(f)
    with open(f'{repo_root}/data/ddb/ddb_relas.json') as f:
        all_relas = json.load(f)
    relas_lst = []
    for key, val in all_relas.items():
        relas_lst.append(val)

    # Ddb pointer to preffered name
    ddb_ptr_to_preferred_name = {}
    # Ddb pointer to name
    ddb_ptr_to_name = defaultdict(list)
    # Ddb name to pointer
    ddb_name_to_ptr = {}

    for key, val in all_names.items():
        item_name = key
        item_ptr = val[0]
        item_preferred = val[1]
        if item_preferred == "1":
            ddb_ptr_to_preferred_name[item_ptr] = item_name
        ddb_name_to_ptr[item_name] = item_ptr
        ddb_ptr_to_name[item_ptr].append(item_name)

    return (relas_lst, ddb_ptr_to_name, ddb_name_to_ptr, ddb_ptr_to_preferred_name)


relas_lst, ddb_ptr_to_name, ddb_name_to_ptr, ddb_ptr_to_preferred_name = load_ddb()

With the extracted names and relationships ONLY for the preferred name entries creates:

- `ddb_ptr_lst`

  A list with each entry being the itemID of the preferred named item

  This will be saved in `"{repo_root}/data/ddb/ptrs.txt"`

- `ddb_names_lst`

  A list with each entry being the item preferred name

  This will be saved in `"{repo_root}/data/ddb/vocab.txt"`


Finally, creates the `id2concept` list which is exactly `ddb_ptr_lst`, so the list of IDs

In [ ]:
ddb_ptr_lst, ddb_names_lst = [], []
for key, val in ddb_ptr_to_preferred_name.items():
    ddb_ptr_lst.append(key)
    ddb_names_lst.append(val)

with open(f"{repo_root}/data/ddb/vocab.txt", "w") as fout:
    for ddb_name in ddb_names_lst:
        print (ddb_name, file=fout)

with open(f"{repo_root}/data/ddb/ptrs.txt", "w") as fout:
    for ddb_ptr in ddb_ptr_lst:
        print (ddb_ptr, file=fout)

id2concept = ddb_ptr_lst

Create the mapping of the relaions in the ddb dataset having the dict relas_dict being:
- Key => The id of the relation in the original dbb dataset
- value => The index of the entry to which it belongs to in the `merged_relations` list

Then, extend it with the relation the pubmed dataset will further provide


In [3]:
merged_relations = [
    'belongs_to_the_category_of',
    'is_a_category',
    'may_cause',
    'is_a_subtype_of',
    'is_a_risk_factor_of',
    'is_associated_with',
    'may_contraindicate',
    'interacts_with',
    'belongs_to_the_drug_family_of',
    'belongs_to_drug_super-family',
    'is_a_vector_for',
    'may_be_allelic_with',
    'see_also',
    'is_an_ingradient_of',
    'may_treat'
]

relas_dict = {"0": 0, "1": 1, "2": 2, "3": 3, "4": 4, "6": 5, "10": 6, "12": 7, "16": 8, "17": 9, "18": 10,
             "20": 11, "26": 12, "30": 13, "233": 14}

metadata_relations = [
'author_in_common',
'mesh_term_in_common',
'chemical_term_in_common',
'same_journal',
'same_year',
'name_entities_with_abstracts_relation',
'god_document_dummy_node_connections',
'qa_context_nodes',
]
metadata_relations_to_id = {w : i+len(merged_relations) for i, w in enumerate(metadata_relations)}

final_merged_relations = merged_relations + metadata_relations

Now, actually build a graph from the triplets of ddb and store it in `"{repo_root}/data/ddb/ddb.graph"`

In [ ]:
import networkx as nx
import pickle

'''
Get:
- concept2id => the dict ItemID:Index
- id2relation => the list of relations
- relation2id => the dict relation:Index

Then, for each relation in ddb get the subj (ID of the source item), obj (ID of
the target item) and the rel (the string explaining the relationship)
Finally, proceede to create the graph
'''
def construct_graph():
    concept2id = {w: i for i, w in enumerate(id2concept)}
    id2relation = merged_relations
    relation2id = {r: i for i, r in enumerate(id2relation)}
    graph = nx.MultiDiGraph()
    attrs = set()
    for relation in relas_lst:
        subj = concept2id[relation[0]]
        obj = concept2id[relation[1]]
        rel = relas_dict[relation[2]]
        weight = 1.
        graph.add_edge(subj, obj, rel=rel, weight=weight)
        attrs.add((subj, obj, rel))
        graph.add_edge(obj, subj, rel=rel + len(final_merged_relations), weight=weight)
        attrs.add((obj, subj, rel + len(final_merged_relations)))
    output_path = f"{repo_root}/data/ddb/ddb.graph"
    #nx.write_gpickle(graph, output_path) deprecated function
    pickle.dump(graph, open(output_path, 'wb'))
    return concept2id, id2relation, relation2id, graph

concept2id, id2relation, relation2id, KG = construct_graph()

# 7) CREATE THE KG OF PUBMED

First, we have to create the metadata relations, so the connection between authors, mesh terms, etc...

So let's first define the method which will help specify if a relation occurs or not

In [ ]:
def encounters(list1, list2):
    return [l for l in list1 if l in list2]

def at_least_N_things_in_common(list1, list2, n):
    if list1 is None:
        list1 = []
    if list2 is None:
        list2 = []
    e = encounters(list1, list2)
    return len(e) >= n

def same_author_1_in_common(authors1, authors2):
    return at_least_N_things_in_common(authors1, authors2, 1)

def same_chemical_term_1_in_common(chemicallist1, chemicallist2):
    return at_least_N_things_in_common(chemicallist1, chemicallist2, 1)

def same_mesh_term_2_in_common(meshtermlist1, meshtermlist2):
    return at_least_N_things_in_common(meshtermlist1, meshtermlist2, 2)

def same_mesh_term_3_in_common(meshtermlist1, meshtermlist2):
    return at_least_N_things_in_common(meshtermlist1, meshtermlist2, 3)

def same_conference(conf1, conf2):
    return conf1 == conf2

def same_year(year1, year2):
    return year1==year2

Now, for each element in `retr_info_ds`, iterate through each element of the feature `retrieval_relative_indices`

For each one of the PUBMED articles linked to that specific question, verify the above relation and, if the relation exists, append the relative triplet

the list of all the relations will be in `json_data`, which will be then saved in `"{medqa_root}/metadata/{fname}_metadata_relations.json"`




In [ ]:
import json
from multiprocessing import Pool
#we only work with the topk documents

def create_metadata_relations(fname):
    json_data = []
    #json_data_names = []
    for i in tqdm(range(len(retr_info_ds[fname]['retrieval_relative_indices']))):
        articles = db_w_metadata[fname][retr_info_ds[fname]['retrieval_relative_indices'][i]]
        articles = dict_of_lists_to_list_of_dicts(articles)
        #articles = articles[:TOPK_DOCUMENTS]

        for doc in articles:
            #json_data_names.append([doc['PMID'], doc['Abstract']])
            for other_doc in articles:
                #common variables
                doc_pmid = doc['PMID']
                other_doc_pmid = other_doc['PMID']
                doc_md = doc['Metadata']
                other_doc_md = other_doc['Metadata']
                #add relations
                if doc_pmid != other_doc_pmid:
                    if same_year(doc['Year'], other_doc['Year']):
                        json_data.append([doc_pmid, other_doc_pmid, metadata_relations_to_id['same_year']])

                    if same_conference(doc['JournalName'], other_doc['JournalName']):
                        json_data.append([doc_pmid, other_doc_pmid, metadata_relations_to_id['same_journal']])

                    if same_author_1_in_common(doc_md['AuthorList'], other_doc_md['AuthorList']):
                        json_data.append([doc_pmid, other_doc_pmid, metadata_relations_to_id['author_in_common']])

                    if same_chemical_term_1_in_common(doc_md['ChemicalTerms'], other_doc_md['ChemicalTerms']):
                        json_data.append([doc_pmid, other_doc_pmid, metadata_relations_to_id['chemical_term_in_common']])

                    if same_mesh_term_3_in_common(doc_md['MeshTerms'], other_doc_md['MeshTerms']):
                        json_data.append([doc_pmid, other_doc_pmid, metadata_relations_to_id['mesh_term_in_common']])


    return json_data#, json_data_names

In [ ]:
#write relations
for fname in FNAMES:
    relations = create_metadata_relations(fname)
    with open(f"{repo_root}/data/{TESTDATASET}/metadata/{fname}_metadata_relations.json", "w") as fout:
        json.dump(relations, fout, ensure_ascii=False)

The same code of above but optimized?

In [ ]:
import json
from multiprocessing import Pool

def create_metadata_relations(indexes):
    json_data = []
    articles = db_w_metadata[FNAME][indexes]
    articles = dict_of_lists_to_list_of_dicts(articles)

    for doc in articles:
        #json_data_names.append([doc['PMID'], doc['Abstract']])
        for other_doc in articles:
            #common variables
            doc_pmid = doc['PMID']
            other_doc_pmid = other_doc['PMID']
            doc_md = doc['Metadata']
            other_doc_md = other_doc['Metadata']
            #add relations
            if doc_pmid != other_doc_pmid:
                if same_year(doc['Year'], other_doc['Year']):
                    json_data.append([doc_pmid, other_doc_pmid, metadata_relations_to_id['same_year']])

                if same_conference(doc['JournalName'], other_doc['JournalName']):
                    json_data.append([doc_pmid, other_doc_pmid, metadata_relations_to_id['same_journal']])

                if same_author_1_in_common(doc_md['AuthorList'], other_doc_md['AuthorList']):
                    json_data.append([doc_pmid, other_doc_pmid, metadata_relations_to_id['author_in_common']])

                if same_chemical_term_1_in_common(doc_md['ChemicalTerms'], other_doc_md['ChemicalTerms']):
                    json_data.append([doc_pmid, other_doc_pmid, metadata_relations_to_id['chemical_term_in_common']])

                if same_mesh_term_3_in_common(doc_md['MeshTerms'], other_doc_md['MeshTerms']):
                    json_data.append([doc_pmid, other_doc_pmid, metadata_relations_to_id['mesh_term_in_common']])


    return json_data


# Example function to process abstract entities in parallel
def process_parallel(function, stmts, num_processes=1):
    # Create a Pool of processes
    with Pool(processes=num_processes, maxtasksperchild=50) as pool:
        # Map the function to statements using the Pool
        processed_stmts = list(tqdm(pool.imap(function, stmts), total=len(stmts)))

    return processed_stmts


batch_stmts = retr_info_ds[FNAME]['retrieval_relative_indices']
processed_batch = process_parallel(create_metadata_relations, batch_stmts, 10)
def flatten(xss):
    return [x for xs in xss for x in xs]
processed = flatten(processed_batch)

In [ ]:
with open(f"{repo_root}/data/{TESTDATASET}/metadata/{FNAME}_metadata_relations.json", "w") as fout:
    json.dump(processed, fout, ensure_ascii=False)

Remove the redundand relations

In [ ]:
import itertools

def remove_redundant_lists(lst):
    unique_relas_set = set(tuple(rel) for rel in lst)  # Use a set of tuples to remove duplicates
    unique_relas = [list(rel) for rel in unique_relas_set]
    return unique_relas

old_len = len(processed)
k = remove_redundant_lists(processed)
removed_files = old_len - len(k)
print(f"file had {old_len} relations, removed {removed_files} relations, remain {len(k)} relations")
with open(f"{repo_root}/data/{TESTDATASET}/metadata/{FNAME}_metadata_relations.json", "w") as fout:
            json.dump(k, fout, ensure_ascii=False)

Now create the acutal pubmed graph

First get again the list of all the relations of pubmed

In [ ]:
def load_metadata_relations(fname):
    with open(f'{repo_root}/data/{TESTDATASET}/metadata/{fname}_metadata_relations.json') as f:
        all_relas = json.load(f)
    #with open(f'{repo_root}/data/{TESTDATASET}/metadata/{fname}_metadata_names.json') as f:
     #   all_names = json.load(f)
    relas_lst = []
    for val in all_relas:
        relas_lst.append(val)

    return relas_lst

md_relas_lst = load_metadata_relations(FNAME)

In [ ]:
import networkx as nx
import pickle


'''
Get:
- concept2id => the dict PMID:Index(it is i+len(id2concept) so like after the
                previous ddb entries(?))
- id2relation => the list of relations
- relation2id => the dict relation:Index

Then, for each relation that we got between the pubmed articles get the subj
(PMID of the source article), obj (PMID of the target article) and the rel (the
string explaining the relationship)
Finally, proceede to create the graph
'''
def construct_metadata_graph():
    concept2id = {w: i+len(id2concept) for i, w in enumerate(md_id2concept)}
    id2relation = metadata_relations
    relation2id = {w : i+len(merged_relations) for i, w in enumerate(id2relation)}
    graph = nx.MultiDiGraph()
    attrs = set()

    for relation in tqdm(md_relas_lst):
        subj = concept2id[str(relation[0])]
        obj = concept2id[str(relation[1])]
        rel = relation[2]
        weight = 1.
        graph.add_edge(subj, obj, rel=rel, weight=weight)
        attrs.add((subj, obj, rel))
        graph.add_edge(obj, subj, rel=rel + len(final_merged_relations), weight=weight)
        attrs.add((obj, subj, rel + len(final_merged_relations)))

    output_path = f"{repo_root}/data/{TESTDATASET}/metadata/metadata.graph"
    #nx.write_gpickle(graph, output_path) deprecated function
    pickle.dump(graph, open(output_path, 'wb'))
    return concept2id, id2relation, relation2id, graph

md_concept2id, md_id2relation, md_relation2id, md_KG = construct_metadata_graph()